# Libraries

In [1]:
import pandas as pd
import pyodbc
import os
from datetime import datetime
from openpyxl import load_workbook
import shutil
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

# Import Data

In [2]:
%%time
week = '19'
#path_so_st = '../../../../../../../../genommalabinternacional/Cristian Javier Sanchez Yepez - NoB2B/Copservir/Copservir Faltante/consolidado_2021.xlsx'
path = '../../../../../../../02Paises/Colombia/Copservir/'

df_sem = pd.read_excel(path + 'Layout/Copservir_2021_' + week + '.xlsx')
df_pos = pd.read_excel(path + 'Maestra Locales Copservir.xlsx', encoding='latin1')
df_prices = pd.read_excel(path + 'Maestra Precios Copservir.xlsx',encoding='latin1')

Wall time: 6.17 s


In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

query_dia = '''SELECT SemFin FROM Gnm_MasterOp.dbo.CatSemanas WHERE SemAnio = {0} AND SemNumero = {1};'''

In [4]:
dia = pd.read_sql(query_dia.format(2021, int(week)), conn)

# Clean Data & Merge

In [5]:
df = df_sem.pivot_table(index=['codHoml', 'Código producto'], values=['Cantidad Saldo', 'so'], aggfunc='sum').reset_index()

df['Fecha_Venta'] = dia.loc[0, 'SemFin']
df['Cadena'] = 'Copservir'

df.rename({'codHoml':'cod', 'Código producto':'Cod_Hom', 'Cantidad Saldo':'Inv'}, axis=1, inplace=True)

In [6]:
df_pos.rename({'PDV':'cod'}, axis=1, inplace=True)

In [7]:
def unique_values(df):
    data = df.copy()
    vals_duplic = data['cod'].value_counts()[data['cod'].value_counts() > 1].index.tolist()
    data_unique_values = data[~data['cod'].isin(vals_duplic)].copy()
    for v in vals_duplic:
        data_unique_values = pd.concat([data_unique_values, data[data['cod'] == v][:1]], axis=0)
    data_unique_values.reset_index(drop=True, inplace=True)
    return data_unique_values

In [8]:
data_pos = unique_values(df_pos)

In [9]:
df[:2]

,cod,Cod_Hom,Inv,so,Fecha_Venta,Cadena
0,101,15706,2.0,0.0,2021-05-09,Copservir
1,101,27543,2.0,1.0,2021-05-09,Copservir


In [10]:
# Cruzamos ambas maestras con los datos de SO y Stock
final = pd.merge(pd.merge(df, data_pos, on='cod', how='left'), df_prices, on='Cod_Hom', how='left')
df.shape[0] == final.shape[0]

True

In [11]:
# Agregamos el Monto de las Ventas
final['Costo_SO'] = final['so']*final['Precio']

# Agregamos el Monto del Stock
final['Costo_Stock'] = final['Inv']*final['Precio']

In [12]:
# Creamos 2 dfs para guardar los datos necesarios

SO = final[['Fecha_Venta', 'Cadena', 'Cod_Prod', 'EAN', 'Descripcion', 'cod', 'NOMBRE PDV', 'so', 'Costo_SO']].copy()

Stock = final[['Fecha_Venta', 'Cadena', 'Cod_Prod', 'EAN', 'Descripcion', 'cod', 'NOMBRE PDV', 'Inv', 'Costo_Stock']].copy()

In [13]:
SO.isnull().sum()

Fecha_Venta    0
Cadena         0
Cod_Prod       0
EAN            0
Descripcion    0
cod            0
NOMBRE PDV     0
so             0
Costo_SO       0
dtype: int64

In [14]:
final['Cod_Hom'][final['Cod_Prod'].isnull()].unique()

array([], dtype=int64)

In [15]:
final['cod'][final['NOMBRE PDV'].isnull()].unique()

array([], dtype=object)

In [16]:
final[final['Cod_Prod'].isnull()]

,cod,Cod_Hom,Inv,so,Fecha_Venta,Cadena,NOMBRE PDV,Cod_Prod,EAN,Descripcion,Precio,Costo_SO,Costo_Stock


In [17]:
Stock.isnull().sum()

Fecha_Venta    0
Cadena         0
Cod_Prod       0
EAN            0
Descripcion    0
cod            0
NOMBRE PDV     0
Inv            0
Costo_Stock    0
dtype: int64

# Export data

In [18]:
def write_layout(filename, df, sellout=True):
    writer = pd.ExcelWriter(filename, engine='openpyxl')
    writer.book = load_workbook(filename)
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    if sellout == True:
        df.to_excel(writer, sheet_name='Ventas', startcol=1, startrow=18, index=False, header=None)
    else:
        df.to_excel(writer, sheet_name='Stock', startcol=1, startrow=18, index=False, header=None)
    writer.save()

In [19]:
%%notify -m "¡Exportación lista!"
%%time
# Creamos un nuevo folder en donde agregamos el layout para exportar a ISV
path_export = '../../../../1Data/2Catalogue/toISV/'
original = path_export + 'Formato No B2B Orig.xlsx'
new_folder = path_export + '/S' + week
try:
    os.mkdir(new_folder)
except:
    pass

# Exportamos los datos al layout de ISV
filename = 'Formato No B2B COPSERVIR 2021_' + str(week).zfill(2)
path_export = new_folder + '/' + filename +'.xlsx'
shutil.copy(original, path_export)
write_layout(path_export, SO, True)
write_layout(path_export, Stock, False)

Wall time: 1min 35s


<IPython.core.display.Javascript object>